In [1]:
from pymongo import MongoClient
import pandas as pd
import json
import re
import os
import csv

In [3]:
# Load the JSON data
data_src = 'cu_paper_data_pub.json'
file_path = os.path.join('.', 'db-data', data_src)

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

def eng_filter(text):
    pattern = r'^[A-Za-z\s.,-]+$'
    return bool(re.match(pattern, text))

result = []
for publication in data:
    if "author_list" in publication and publication["author_list"]:
        # base_affiliation = publication["author_list"][0]["affiliation"]  (Use the first author's affiliation as the base)
        if publication["subject code"]:
            for code in publication["subject code"]:
                cleaned_authors = [
                    {
                        "title": publication["title"],
                        "name": author["author"],
                        "affiliation": author["affiliation"], 
                        "subject_code": code,  
                        "cited_count": publication["cited_count"],
                        "pub_type": publication["publication_type"],
                        "year": publication["year"]
                    }
                    for author in publication["author_list"]
                    if eng_filter(author["author"]) 
                ]
                result.extend(cleaned_authors)

# Define the CSV file name
csv_name = 'clean_cu_paper_data_pub.csv'
keys = result[0].keys()

# Write the flattened data to CSV
with open(csv_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=keys)
    # Write header (field names)
    writer.writeheader()
    # Write data rows
    writer.writerows(result)
print(f"Data has been saved to '{csv_name}'")

Data has been saved to 'clean_cu_paper_data_pub.csv'
